# Minecraft Stats

This notebook summarize minecraft stats files across all worlds saved on the local machine. PathLib is used - configure pathlib to work with your operating system.

The first code cell (immediately below) builds the 

In [87]:
import pandas as pd
import re
from pathlib import Path

local_user = 'culley'
# OSX path
path = Path('/Users/{user}/Library/Application Support/minecraft/saves/'.format(user=local_user))
files = list(path.rglob("*/stats/*.json"))


def minecraft_key(key_name):
    return 'minecraft:{key}'.format(key=key_name)

def merge_data_frames(df, df2, df_name):
    return pd.merge(
        df,
        pd.DataFrame({'minecraft_key':df2.keys(), df_name: df2.values()}),
        how="left",
        left_on="minecraft_key",
        right_on="minecraft_key"
    )

def world_stats(path):
    df = pd.read_json(path)
    stats = df['stats']
    try:
        broken = stats[minecraft_key('broken')]
    except KeyError:
        broken = {}
    try:
        crafted = stats[minecraft_key('crafted')]
    except KeyError:
        crafted = {}
    try:
        custom = stats[minecraft_key('custom')]
    except KeyError:
        custom = {}       
    try:
        dropped = stats[minecraft_key('dropped')]
    except KeyError:
        dropped = {}
    try:
        killed = stats[minecraft_key('killed')]
    except KeyError:
        killed = {}
    try:
        mined = stats[minecraft_key('mined')]
    except KeyError:
        mined = {}           
    try:
        picked_up = stats[minecraft_key('picked_up')]
    except KeyError:
        picked_up = {}
    try:
        used = stats[minecraft_key('used')]
    except KeyError:
        used = {}     
        
    df = pd.DataFrame()
    # create unique list of keys from all dictionaries
    df['minecraft_key'] = list(set(broken.keys()) | 
                          set(crafted.keys()) | 
                          set(custom.keys()) | 
                          set(dropped.keys()) | 
                          set(killed.keys()) | 
                          set(mined.keys()) | 
                          set(picked_up.keys()) | 
                          set(used.keys()))
    df = merge_data_frames(df, broken, 'broken')
    df = merge_data_frames(df, crafted, 'crafted')
    df = merge_data_frames(df, custom, 'custom')
    df = merge_data_frames(df, dropped, 'dropped')
    df = merge_data_frames(df, killed, 'killed')
    df = merge_data_frames(df, mined, 'mined')
    df = merge_data_frames(df, picked_up, 'picked_up')
    df = merge_data_frames(df, used, 'used')
    # remove minecraft: prefix
    df['minecraft_key'] =  [re.sub(r'minecraft:','', str(x)) for x in df['minecraft_key']]
    df['world_name'] = path.parts[7]
    df['wood_type'] = df['minecraft_key'].str.extract(r'(dark_oak|birch|oak|acacia|spruce|jungle|mangrove)')
    #df.set_index('minecraft_key')
    df = df.fillna(0)
    df = df.astype({'broken': 'int','broken': 'int','crafted': 'int','custom': 'int','dropped': 'int','killed': 'int','mined': 'int','picked_up': 'int','used': 'int'})
    return df

df = pd.DataFrame()
for file_name in files:
    try:
        df = pd.concat([df, world_stats(file_name)])
    except ValueError:
        print('Value Error:  ', file_name.as_uri())
    except KeyError:
        print('Key Error:  ', file_name.as_uri())



Key Error:   file:///Users/culley/Library/Application%20Support/minecraft/saves/Nell%27s%20World/stats/cbe07d0e-9005-4de5-8809-05c52ae494ef.json
Key Error:   file:///Users/culley/Library/Application%20Support/minecraft/saves/Nell%27s%20World/stats/9b1b448b-b6f4-4fb4-8920-9ddce220459c.json
Value Error:   file:///Users/culley/Library/Application%20Support/minecraft/saves/the%20big%20one/stats/cbe07d0e-9005-4de5-8809-05c52ae494ef.json
Value Error:   file:///Users/culley/Library/Application%20Support/minecraft/saves/the%20big%20one/stats/acdedf37-063b-4eb9-ba39-79cee56ac764.json
Value Error:   file:///Users/culley/Library/Application%20Support/minecraft/saves/The%20Field/stats/cbe07d0e-9005-4de5-8809-05c52ae494ef.json
Key Error:   file:///Users/culley/Library/Application%20Support/minecraft/saves/New%20World--/stats/acdedf37-063b-4eb9-ba39-79cee56ac764.json
Key Error:   file:///Users/culley/Library/Application%20Support/minecraft/saves/New%20World--/stats/9b1b448b-b6f4-4fb4-8920-9ddce22045

In [107]:
import hvplot.pandas

woods = df.groupby('wood_type').sum()
woods['id'] = woods.index
woods = woods.drop(woods.index[0])
woods_long_format = pd.melt(woods, id_vars=['id'], value_vars=['crafted', 'mined', 'picked_up', 'used'])
woods_long_format.hvplot.bar('id', 'value', 
                             by='variable', 
                             legend='top_left', 
                             height=500, 
                             width=1000,
                             ylabel='Total',
                             xlabel='Wood Type Statistics',
                             rot=60, 
                             title='Wood Type Summary Statistics for all Saved Worlds')

:Bars   [id,variable]   (value)